# Stats from the Titles Collection

This notebook looks at some stats from the Titles collection. The first block returns all titles held by an individual, in this case Charlemagne.

In [1]:
from pymongo import MongoClient
import pandas as pd
import datetime

In [2]:
client = MongoClient()
titles = client.ck2.titles

## All Titles held by Charlemagne

In [3]:
pipeline = [
    {
        "$unwind" : "$holders"        
    },
    {
        "$lookup": 
        {
            "from" : "characters",
            "localField" : "holders.holder",
            "foreignField" : "_id",
            "as" : "title_holder"
        }
    },
    {
        "$unwind" : "$title_holder"
    },
    {
        "$match" : {"title_holder._id" : 6392}
    },
    {
        "$project" : {"_id" : 0, "title": 1, "holders.start_date" :1, "holders.end_date" : 1}
    }
]

In [4]:
for char in titles.aggregate(pipeline):
    print(char)

{'title': 'd_gelre', 'holders': {'start_date': datetime.datetime(775, 7, 7, 0, 0), 'end_date': datetime.datetime(801, 10, 19, 0, 0)}}
{'title': 'c_gelre', 'holders': {'start_date': datetime.datetime(769, 1, 1, 0, 0), 'end_date': datetime.datetime(801, 10, 19, 0, 0)}}
{'title': 'c_frisia', 'holders': {'start_date': datetime.datetime(769, 1, 1, 0, 0), 'end_date': datetime.datetime(801, 10, 19, 0, 0)}}
{'title': 'k_lotharingia', 'holders': {'start_date': datetime.datetime(771, 2, 12, 0, 0), 'end_date': datetime.datetime(778, 6, 24, 0, 0)}}
{'title': 'k_lotharingia', 'holders': {'start_date': datetime.datetime(791, 7, 8, 0, 0), 'end_date': datetime.datetime(801, 10, 19, 0, 0)}}
{'title': 'd_luxembourg', 'holders': {'start_date': datetime.datetime(772, 5, 7, 0, 0), 'end_date': datetime.datetime(772, 5, 8, 0, 0)}}
{'title': 'c_liege', 'holders': {'start_date': datetime.datetime(796, 5, 30, 0, 0), 'end_date': datetime.datetime(801, 10, 19, 0, 0)}}
{'title': 'd_upper_lorraine', 'holders': {'st

## Longest Held Elective Kindom or Empire Titles

Title holders are stored in the save file with how they were aquired; through inheritance, conquest or election. Below are the Kingdom and Empire level titles that individuals were elected to ordered by the longest reign. Counties and Duchies are excluded as often a child will be elected on their fathers death while Kingdom and Empire titles are more selective. 

The figure for reign length is in days. Mamoudou Cisse led Mali for almost 48 years. Callistus was the longest reigning pope at 35 years.

In [5]:
pipeline = [
    {
        "$match" : {"$or" : [{"title" : {"$regex" : '^k_.*'}}, {"title" : {"$regex" : '^e_.*'}}]}
    },
    {
        "$unwind" : "$holders"        
    },
    {
        "$lookup": 
        {
            "from" : "characters",
            "localField" : "holders.holder",
            "foreignField" : "_id",
            "as" : "title_holder"
        }
    },
    {
        "$unwind" : "$title_holder"
    },
    {
        "$lookup": 
        {
            "from" : "dynasties",
            "localField" : "title_holder.dnt",
            "foreignField" : "_id",
            "as" : "title_holder_dyn"
        }
    },
    {
        "$unwind" : "$title_holder_dyn"
    },
    
    {
        "$match" : {"holders.type" : "election"}
    },
    {
        "$project" : {"_id" : 0, 'title' : 1, 
                      "name" : {"$ifNull" : ["$title_holder.taken_name", {"$concat" : ["$title_holder.bn", " ", "$title_holder_dyn.name" ]}]},
                     "reign_length" : "$holders.reign_length", "start_date" : "$holders.start_date", "end_date" : "$holders.end_date", 
                      "d_d" : "$title_holder.d_d"}
    },
    {
        "$sort" : {"reign_length" : -1}
    },
    {
        "$limit" : 25
    }
]

In [6]:
for title in titles.aggregate(pipeline):
    if "end_date" in title.keys() and title["d_d"] >= title["end_date"]:
        print(title)

{'title': 'k_mali', 'name': 'Mamoudou Cisse', 'reign_length': 17498, 'start_date': datetime.datetime(1069, 7, 10, 0, 0), 'end_date': datetime.datetime(1117, 6, 7, 0, 0), 'd_d': datetime.datetime(1117, 6, 7, 0, 0)}
{'title': 'k_poland', 'name': 'Cirulis Palemonaitis', 'reign_length': 16715, 'start_date': datetime.datetime(817, 11, 15, 0, 0), 'end_date': datetime.datetime(863, 8, 21, 0, 0), 'd_d': datetime.datetime(863, 8, 21, 0, 0)}
{'title': 'k_bulgaria', 'name': 'Butigeidis Palemonaitis', 'reign_length': 16430, 'start_date': datetime.datetime(869, 6, 20, 0, 0), 'end_date': datetime.datetime(914, 6, 15, 0, 0), 'd_d': datetime.datetime(914, 6, 15, 0, 0)}
{'title': 'k_saxony', 'name': 'Hesso Hessing', 'reign_length': 14951, 'start_date': datetime.datetime(797, 10, 22, 0, 0), 'end_date': datetime.datetime(838, 9, 28, 0, 0), 'd_d': datetime.datetime(838, 9, 28, 0, 0)}
{'title': 'k_finland', 'name': 'Dobrinya Saaremaa', 'reign_length': 14812, 'start_date': datetime.datetime(1202, 2, 23, 0, 

# Most County Titles Held, most Dukedoms, most Kingdoms and Empires etc

In [7]:
pipeline = [
    {
        "$match" : {"title" : {"$regex" : '^c_.*'}}
    },
    {
        "$unwind" : "$holders"        
    },
    {
        "$lookup": 
        {
            "from" : "characters",
            "localField" : "holders.holder",
            "foreignField" : "_id",
            "as" : "title_holder"
        }
    },
    {
        "$unwind" : "$title_holder"
    },
    {
        "$lookup": 
        {
            "from" : "dynasties",
            "localField" : "title_holder.dnt",
            "foreignField" : "_id",
            "as" : "title_holder_dyn"
        }
    },
    {
        "$unwind" : "$title_holder_dyn"
    },    
    {
        "$group" : {"_id" : "$holders.holder",
                    "firstname" : {"$first" : "$title_holder.bn"},  
                    "surname" : {"$first" : "$title_holder_dyn.name"},
                    "titles" : {"$addToSet": "$title"},
                    "d_d" : {"$first" : "$title_holder.d_d"}
                    },
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]},  
                      "title_count" : {"$size" : "$titles"}, "d_d" : "$d_d"}
    },
    {
        "$sort" : {"title_count" : -1}
    },
    {
        "$limit" : 20
    }
]

In [8]:
for title in titles.aggregate(pipeline):
    print(title)

{'_id': 786263, 'name': 'Eadulf Mel', 'title_count': 37, 'd_d': datetime.datetime(1195, 2, 24, 0, 0)}
{'_id': 635388, 'name': 'Madanapala Ayudha', 'title_count': 33, 'd_d': datetime.datetime(1025, 8, 13, 0, 0)}
{'_id': 932125, 'name': 'Qawurd Najibid', 'title_count': 32, 'd_d': datetime.datetime(1372, 9, 7, 0, 0)}
{'_id': 725343, 'name': 'Theocharistos Komnenodoukas', 'title_count': 32, 'd_d': datetime.datetime(1133, 9, 4, 0, 0)}
{'_id': 512268, 'name': 'Isa Isaid', 'title_count': 32, 'd_d': datetime.datetime(839, 9, 9, 0, 0)}
{'_id': 756903, 'name': 'Markos Komnenodoukas', 'title_count': 31, 'd_d': datetime.datetime(1135, 4, 8, 0, 0)}
{'_id': 758926, 'name': 'Eusebios Komnenodoukas', 'title_count': 31, 'd_d': datetime.datetime(1138, 7, 15, 0, 0)}
{'_id': 890982, 'name': 'Abdullah Najibid', 'title_count': 30, 'd_d': datetime.datetime(1336, 6, 23, 0, 0)}
{'_id': 944090, 'name': 'Maccus Clifford', 'title_count': 29, 'd_d': datetime.datetime(1415, 3, 15, 0, 0)}
{'_id': 908627, 'name': 'Ev

This list shows the rulers who held the most amount of counties. Eadulf was the King of England who won the Crusade for the Greek Republic after it elected a series of Islamic rulers. While he held 37 unique county titles during his life many of these may have been held for less than a day before redistributing them after the conquest of Greece. 
Bishop Theocharistos founded the Republic of Greece and while no other member of the Komnenodoukas Dynasty succeded him to that title his large holdings were to his distant relative Bishop Markos.
The Knights Hospitaller controlled most of Germany and a number of their leaders appear above; the father and son Ordgar and Maccus Clifford, Evrard de Flotte, Alain Nibelunging, Aldebert de Trith and Totil z Roztoki.

## Most Duchies

In [9]:
pipeline = [
    {
        "$match" : {"title" : {"$regex" : '^d_.*'}}
    },
    {
        "$unwind" : "$holders"        
    },
    {
        "$lookup": 
        {
            "from" : "characters",
            "localField" : "holders.holder",
            "foreignField" : "_id",
            "as" : "title_holder"
        }
    },
    {
        "$unwind" : "$title_holder"
    },
    {
        "$lookup": 
        {
            "from" : "dynasties",
            "localField" : "title_holder.dnt",
            "foreignField" : "_id",
            "as" : "title_holder_dyn"
        }
    },
    {
        "$unwind" : "$title_holder_dyn"
    },    
    {
        "$group" : {"_id" : "$holders.holder",
                    "firstname" : {"$first" : "$title_holder.bn"},  
                    "surname" : {"$first" : "$title_holder_dyn.name"},
                    "titles" : {"$addToSet": "$title"},
                    "d_d" : {"$first" : "$title_holder.d_d"}
                    },
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]},  
                      "title_count" : {"$size" : "$titles"}, "d_d" : "$d_d",
                      "titles" : "$titles"
                     },
                        
    },
    {
        "$sort" : {"title_count" : -1}
    },
    {
        "$limit" : 1
    }
]

In [10]:
for title in titles.aggregate(pipeline):
    print(title)

{'_id': 832705, 'name': 'Timotheos Kritopolos', 'title_count': 33, 'd_d': datetime.datetime(1273, 9, 14, 0, 0), 'titles': ['d_pecs', 'd_wallachia', 'd_khiva', 'd_syr_darya', 'd_turkestan', 'd_upper_burgundy', 'd_kiev', 'd_dalmatia', 'd_silesia', 'd_rashka', 'd_sardinia', 'd_karvuna', 'd_bosnia', 'd_turnovo', 'd_vidin', 'd_charsianon', 'd_edessa', 'd_alania', 'd_cibyrrhaeot', 'd_samos', 'd_athens', 'd_paphlagonia', 'd_adrianopolis', 'd_moravia', 'd_armenia_minor', 'd_nikaea', 'd_osterreich', 'd_armenia', 'd_coloneia', 'd_carinthia', 'd_dyrrachion', 'd_tyrol', 'd_bavaria']}


Replacing a distant relative, Timotheos Kritopolos was installed as leader of Byzantium and may possibly have revoked and redistributed many of these titles.

## Most Kingdoms/Empires

In [11]:
pipeline = [
    {
        "$match" : {"$or" : [{"title" : {"$regex" : '^k_.*'}}, {"title" : {"$regex" : '^e_.*'}}]}
    },
    {
        "$unwind" : "$holders"        
    },
    {
        "$lookup": 
        {
            "from" : "characters",
            "localField" : "holders.holder",
            "foreignField" : "_id",
            "as" : "title_holder"
        }
    },
    {
        "$unwind" : "$title_holder"
    },
    {
        "$lookup": 
        {
            "from" : "dynasties",
            "localField" : "title_holder.dnt",
            "foreignField" : "_id",
            "as" : "title_holder_dyn"
        }
    },
    {
        "$unwind" : "$title_holder_dyn"
    },    
    {
        "$group" : {"_id" : "$holders.holder",
                    "firstname" : {"$first" : "$title_holder.bn"},  
                    "surname" : {"$first" : "$title_holder_dyn.name"},
                    "titles" : {"$addToSet": "$title"},
                    "d_d" : {"$first" : "$title_holder.d_d"}
                    },
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]},  
                      "title_count" : {"$size" : "$titles"}, "d_d" : "$d_d",
                      "titles" : "$titles"
                     },
                        
    },
    {
        "$sort" : {"title_count" : -1}
    },
    {
        "$limit" : 1
    }
]

In [12]:
for title in titles.aggregate(pipeline):
    print(title)

{'_id': 1034299, 'name': 'Kanvarpala Vidurathpalid', 'title_count': 8, 'd_d': None, 'titles': ['k_maharastra', 'k_orissa', 'k_bengal', 'k_kosala', 'k_rajputana', 'k_delhi', 'k_telingana', 'k_punjab']}


14 years at the end at the games end Kanvarpala holds the same 8 kingdoms held by his father and grandfather.